<a href="https://colab.research.google.com/github/mlacasa/NLTK/blob/main/C%C3%A0lcul_Contingency_Table_N_Grams_NLTK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Carreguem les llibreries
import nltk
nltk.download('all')
import pandas as pd
import gensim
from collections import Counter
import  re
from nltk import word_tokenize, pos_tag
from nltk import word_tokenize
from nltk.util import ngrams
from nltk.collocations import *
import re

In [2]:
# Carreguem una dataset de Kaggle https://www.kaggle.com/ankurzing/sentiment-analysis-for-financial-news

df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Mineria Textos/PAC1/all-data.csv',encoding = "ISO-8859-1", header=None,   names=['Sentiment','News'])

In [3]:
df.head()

,Sentiment,News
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [4]:
# Es un problema de classificació
df['Sentiment'].value_counts()

neutral     2879
positive    1363
negative     604
Name: Sentiment, dtype: int64

In [5]:
# No hi ha valors nulls
df.isna().sum()

Sentiment    0
News         0
dtype: int64

In [6]:
# Pasem news a un llistat

sentences = [st for st in df['News']]

# i els pasem a un sol tex amb minúscula

sentences = " ".join(sentences).lower()

In [7]:
# Tokenizer

tokens = [w for w in word_tokenize(sentences)]

In [8]:
# Filrem per stopwords i per signes de puntuació.
from nltk.corpus import stopwords
stopwords_ = set(stopwords.words('english'))
tokens_clean = [word.lower() for word in tokens if word not in stopwords_]
tokens_clean = [w for w in tokens_clean if not re.match(r'\W',  w)]

In [9]:
# Etiquetem les paraules

tokens = nltk.pos_tag(tokens_clean)

In [10]:
# Exemple
tokens[:5]

[('according', 'VBG'),
 ('gran', 'NN'),
 ('company', 'NN'),
 ('plans', 'VBZ'),
 ('move', 'VBP')]

In [11]:
# Creo els objectes bigrams

from nltk import bigrams
bigram_measures = nltk.collocations.BigramAssocMeasures()

In [12]:
# Definim els bigrams del text
b = BigramCollocationFinder.from_words(tokens)

In [13]:
# Nº paraules utilitzades.
b.word_fd.N()

63586

In [14]:
# Freqüència relativa de cada bigram
b.score_ngrams(bigram_measures.raw_freq)[:10]

[((('net', 'JJ'), ('sales', 'NNS')), 0.003475607838203378),
 ((('operating', 'NN'), ('profit', 'NN')), 0.002704997955524801),
 ((('net', 'JJ'), ('profit', 'NN')), 0.001399679174661089),
 ((('said', 'VBD'), ('today', 'NN')), 0.0013524989777624005),
 ((('mln', 'NN'), ('euro', 'NN')), 0.001242411851665461),
 ((('per', 'IN'), ('share', 'NN')), 0.0011795049224672097),
 ((('corresponding', 'VBG'), ('period', 'NN')), 0.0011008712609693958),
 ((('mn', 'NN'), ('eur', 'NN')), 0.001085144528669833),
 ((('company', 'NN'), ('said', 'VBD')), 0.00106941779637027),
 ((('profit', 'NN'), ('eur', 'NN')), 0.0009436039379737678)]

Al llibre Foundations of Statistical Natural Language Processing\
Al tema 5\
Pel càlcul de la mètrica PMI indica

Pointwise Mutual information (PMI) té com a fórmula:
$$PMI(a,b) = log_{2}\frac{P(a,b)}{P(a) P(b)}$$
on $a$ i $b$ son indepentents. A valors propers a 1 vol dir que la seva concurrensa és casual. Llavors el $log(1) = 0$ voldria dir que valors $PMI = 0$ la seva concurrència és casual. 

Pero els resultats no quadren així que mirem la [documentació oficial](https://www.nltk.org/_modules/nltk/metrics/association.html) per veure la fórmula emprada:

```
    @classmethod
    def pmi(cls, *marginals):
        """Scores ngrams by pointwise mutual information, as in Manning and
        Schutze 5.4.
        """
        return _log2(marginals[NGRAM] * marginals[TOTAL] ** (cls._n - 1)) - _log2(
            _product(marginals[UNIGRAMS])
        )

```
I la funció likelihood_ratio és:


```
    def likelihood_ratio(cls, *marginals):
        """Scores ngrams using likelihood ratios as in Manning and Schutze 5.3.4."""
        cont = cls._contingency(*marginals)
        return 2 * sum(
            obs * _ln(obs / (exp + _SMALL) + _SMALL)
            for obs, exp in zip(cont, cls._expected_values(cont))
        )

```
on 



```
class BigramAssocMeasures(NgramAssocMeasures)
 |  A collection of bigram association measures. Each association measure
 |  is provided as a function with three arguments::
 |  
 |      bigram_score_fn(n_ii, (n_ix, n_xi), n_xx)
 |  
 |  The arguments constitute the marginals of a contingency table, counting
 |  the occurrences of particular events in a corpus. The letter i in the
 |  suffix refers to the appearance of the word in question, while x indicates
 |  the appearance of any word. Thus, for example:
 |  
 |      n_ii counts (w1, w2), i.e. the bigram being scored
 |      n_ix counts (w1, *)
 |      n_xi counts (*, w2)
 |      n_xx counts (*, *), i.e. any bigram
 |  
 |  This may be shown with respect to a contingency table::
 |  
 |              w1    ~w1
 |           ------ ------
 |       w2 | n_ii | n_oi | = n_xi
 |           ------ ------
 |      ~w2 | n_io | n_oo |
 |           ------ ------
 |           = n_ix        TOTAL = n_xx
 |  
```

On si fem les traduccions:

$$marginals[NGRAM] = n_{ii}$$
$$marginals[TOTAL] = n_{xx}$$
$$marginals[UNIGRAMS] = (n_{xi}, n_{ix})$$


In [15]:
# No he trobat manera de calcular les contingències, però les podem calcular a mà.
# Utilitzem les dades de freqüències relatives per calcular les freqüències absolutes
df1= pd.DataFrame(b.score_ngrams(bigram_measures.raw_freq))
df1.head()

,0,1
0,"((net, JJ), (sales, NNS))",0.003476
1,"((operating, NN), (profit, NN))",0.002705
2,"((net, JJ), (profit, NN))",0.001400
3,"((said, VBD), (today, NN))",0.001352
4,"((mln, NN), (euro, NN))",0.001242


In [16]:
df1['w1'] = [x[0][0] for x in df1.iloc[:,0]]
df1['w1type'] = [x[0][1] for x in df1.iloc[:,0]]
df1['w2'] = [x[1][0] for x in df1.iloc[:,0]]
df1['w2type'] = [x[1][1] for x in df1.iloc[:,0]]
df1.head()

,0,1,w1,w1type,w2,w2type
0,"((net, JJ), (sales, NNS))",0.003476,net,JJ,sales,NNS
1,"((operating, NN), (profit, NN))",0.002705,operating,NN,profit,NN
2,"((net, JJ), (profit, NN))",0.001400,net,JJ,profit,NN
3,"((said, VBD), (today, NN))",0.001352,said,VBD,today,NN
4,"((mln, NN), (euro, NN))",0.001242,mln,NN,euro,NN


In [17]:
# Calculem les mètriques del primer bigram - (net, sales)
# Si fem w1 = (net, JJ) i w2 = (sales, NNS)
df1[df1['w1'] == 'net']

,0,1,w1,w1type,w2,w2type
0,"((net, JJ), (sales, NNS))",0.003476,net,JJ,sales,NNS
2,"((net, JJ), (profit, NN))",0.001400,net,JJ,profit,NN
14,"((net, JJ), (loss, NN))",0.000739,net,JJ,loss,NN
53,"((net, NN), (sales, NNS))",0.000393,net,NN,sales,NNS
211,"((net, JJ), (interest, NN))",0.000157,net,JJ,interest,NN
611,"((net, JJ), (income, NN))",0.000079,net,JJ,income,NN
916,"((net, NN), (profit, NN))",0.000063,net,NN,profit,NN
4021,"((net, JJ), (cash, NN))",0.000031,net,JJ,cash,NN
33005,"((net, JJ), (debt, NN))",0.000016,net,JJ,debt,NN
33006,"((net, JJ), (investment, NN))",0.000016,net,JJ,investment,NN


In [18]:
# Per la cerca necessitem exigir que w1type sigui JJ
df1[(df1['w1'] == 'net') & (df1['w1type'] == 'JJ')]

,0,1,w1,w1type,w2,w2type
0,"((net, JJ), (sales, NNS))",0.003476,net,JJ,sales,NNS
2,"((net, JJ), (profit, NN))",0.001400,net,JJ,profit,NN
14,"((net, JJ), (loss, NN))",0.000739,net,JJ,loss,NN
211,"((net, JJ), (interest, NN))",0.000157,net,JJ,interest,NN
611,"((net, JJ), (income, NN))",0.000079,net,JJ,income,NN
4021,"((net, JJ), (cash, NN))",0.000031,net,JJ,cash,NN
33005,"((net, JJ), (debt, NN))",0.000016,net,JJ,debt,NN
33006,"((net, JJ), (investment, NN))",0.000016,net,JJ,investment,NN
33007,"((net, JJ), (losses, NNS))",0.000016,net,JJ,losses,NNS
33008,"((net, JJ), (operating, NN))",0.000016,net,JJ,operating,NN


In [19]:
# Però net pot ser a la columna w2?
df1[(df1['w2'] == 'net') & (df1['w2type'] == 'JJ')]

,0,1,w1,w1type,w2,w2type
51,"((today, NN), (net, JJ))",0.000409,today,NN,net,JJ
70,"((expects, VBZ), (net, JJ))",0.000299,expects,VBZ,net,JJ
102,"((said, VBD), (net, JJ))",0.000252,said,VBD,net,JJ
146,"((reports, NNS), (net, JJ))",0.000204,reports,NNS,net,JJ
184,"((profit, NN), (net, JJ))",0.000173,profit,NN,net,JJ
...,...,...,...,...,...,...
48253,"((usd1, JJ), (net, JJ))",0.000016,usd1,JJ,net,JJ
48480,"((vaisala, JJ), (net, JJ))",0.000016,vaisala,JJ,net,JJ
48711,"((versus, NN), (net, JJ))",0.000016,versus,NN,net,JJ
49208,"((wednesday, JJ), (net, JJ))",0.000016,wednesday,JJ,net,JJ


In [20]:
#Hem de tenir en compte les dues possibilitats
df1[(df1['w1'] == 'net') & (df1['w1type'] == 'JJ') | (df1['w2'] == 'net') & (df1['w2type'] == 'JJ')]

,0,1,w1,w1type,w2,w2type
0,"((net, JJ), (sales, NNS))",0.003476,net,JJ,sales,NNS
2,"((net, JJ), (profit, NN))",0.001400,net,JJ,profit,NN
14,"((net, JJ), (loss, NN))",0.000739,net,JJ,loss,NN
51,"((today, NN), (net, JJ))",0.000409,today,NN,net,JJ
70,"((expects, VBZ), (net, JJ))",0.000299,expects,VBZ,net,JJ
...,...,...,...,...,...,...
48253,"((usd1, JJ), (net, JJ))",0.000016,usd1,JJ,net,JJ
48480,"((vaisala, JJ), (net, JJ))",0.000016,vaisala,JJ,net,JJ
48711,"((versus, NN), (net, JJ))",0.000016,versus,NN,net,JJ
49208,"((wednesday, JJ), (net, JJ))",0.000016,wednesday,JJ,net,JJ


In [21]:
# Ara podem calcular les freqüències relatives de:
# 1- P(w1,w2) = n_ii = marginals[NGRAM]

n_ii = df1.iloc[0,1]
n_ii

0.003475607838203378

In [22]:
# Tamnbé podem calcular la freüència relativa de P(w1,*) que és:
# 2- P(w1,*) = n_ix = marginals[UNIGRAM] però un dels dos elements.
# sumem la columna de les freq relatives de l'anterior consulta.
n_xi = df1[(df1['w1'] == 'net') & (df1['w1type'] == 'JJ') | (df1['w2'] == 'net') & (df1['w2type'] == 'JJ')].iloc[:,1].sum()
n_xi

0.011983770012266848

In [23]:
# Ara hem de calcular n_ix amb la paraula (sales,NNS)
n_ix = df1[(df1['w1'] == 'sales') & (df1['w1type'] == 'NNS') | (df1['w2'] == 'sales') & (df1['w2type'] == 'NNS')].iloc[:,1].sum()
n_ix

0.014248419463403893

In [24]:
# Ara ens queda calcular el total. Hipòtesi, la última fila de df1 és de freqüència absoluta 1.
b.apply_freq_filter(2) # bf conté tots els bigrams amb freqüència min 2

In [25]:
# a bf no son la freqüència 1, si la meva hipòtesis és correcte, hi haurà menys files a la dataframe:

df2= pd.DataFrame(b.score_ngrams(bigram_measures.raw_freq))
df2.head()

,0,1
0,"((net, JJ), (sales, NNS))",0.003476
1,"((operating, NN), (profit, NN))",0.002705
2,"((net, JJ), (profit, NN))",0.001400
3,"((said, VBD), (today, NN))",0.001352
4,"((mln, NN), (euro, NN))",0.001242


In [26]:
print('Sense filtrar hi ha : ',df1.shape[0],' bigrams analitzats i filtrats (eliminem freq 1): ',df2.shape[0])

Sense filtrar hi ha :  50264  bigrams analitzats i filtrats (eliminem freq 1):  5296


això vol dir que 45.000 de 50.000 bigrams tenen freqüència absoluta 1.

In [27]:
df1.iloc[6000:6010,:]

,0,1,w1,w1type,w2,w2type
6000,"((133.4, CD), (kroner, NN))",0.000016,133.4,CD,kroner,NN
6001,"((134, CD), (million, CD))",0.000016,134,CD,million,CD
6002,"((134, CD), (mln, NN))",0.000016,134,CD,mln,NN
6003,"((134,700, CD), (company, NN))",0.000016,"134,700",CD,company,NN
6004,"((135, CD), (million, CD))",0.000016,135,CD,million,CD
6005,"((135, CD), (sek, NN))",0.000016,135,CD,sek,NN
6006,"((135-gram, JJ), (model, NN))",0.000016,135-gram,JJ,model,NN
6007,"((135.7, CD), (mn, NN))",0.000016,135.7,CD,mn,NN
6008,"((137, CD), (million, CD))",0.000016,137,CD,million,CD
6009,"((137.3, CD), (given, VBN))",0.000016,137.3,CD,given,VBN


In [28]:
# Llavors podem calcular la freqüència absoluta total (la que ens manca) per calcular les freqüències absolutes totals.

n_xx = 1/df1.iloc[6000,1]
n_xx

63586.0

In [29]:
# Ara podem copmletar les freqüències absolutes

n_ii = n_ii * n_xx
n_ix = n_ix * n_xx
n_xi = round(n_xi * n_xx,0)

In [30]:
print('marginals[TOTAL] = ',n_xx, ' marginals[NGRAM] = ',n_ii,' marginals[UNIGRAM] = (',n_ix,',',n_xi,')')

marginals[TOTAL] =  63586.0  marginals[NGRAM] =  221.0  marginals[UNIGRAM] = ( 906.0 , 762.0 )


In [31]:
# Càlcul PMI

b = BigramCollocationFinder.from_words(tokens)

In [32]:
df_pmi = pd.DataFrame(b.score_ngrams(bigram_measures.pmi))

In [33]:
df_pmi.head()

,0,1
0,"((0.31, CD), (emsa, NN))",15.956422
1,"((03-04, JJ), (xvi, JJ))",15.956422
2,"((03.02.2011, CD), (unimilk, JJ))",15.956422
3,"((04.01.2006, CD), (13.00, CD))",15.956422
4,"((0762, CD), (shse, NN))",15.956422


In [34]:
df_pmi['w1'] = [x[0][0] for x in df_pmi.iloc[:,0]]
df_pmi['w1type'] = [x[0][1] for x in df_pmi.iloc[:,0]]
df_pmi['w2'] = [x[1][0] for x in df_pmi.iloc[:,0]]
df_pmi['w2type'] = [x[1][1] for x in df_pmi.iloc[:,0]]
df_pmi.head()

,0,1,w1,w1type,w2,w2type
0,"((0.31, CD), (emsa, NN))",15.956422,0.31,CD,emsa,NN
1,"((03-04, JJ), (xvi, JJ))",15.956422,03-04,JJ,xvi,JJ
2,"((03.02.2011, CD), (unimilk, JJ))",15.956422,03.02.2011,CD,unimilk,JJ
3,"((04.01.2006, CD), (13.00, CD))",15.956422,04.01.2006,CD,13.00,CD
4,"((0762, CD), (shse, NN))",15.956422,0762,CD,shse,NN


In [35]:
df_pmi[(df_pmi['w1'] == 'net') & (df_pmi['w2'] == 'sales') ]

,0,1,w1,w1type,w2,w2type
35730,"((net, NN), (sales, NNS))",6.822714,net,NN,sales,NNS
37740,"((net, JJ), (sales, NNS))",6.347310,net,JJ,sales,NNS


In [36]:
# El nostre bigram té un PMI calculat de 6.347310

Recordem la fórmula

```
    @classmethod
    def pmi(cls, *marginals):
        """Scores ngrams by pointwise mutual information, as in Manning and
        Schutze 5.4.
        """
        return _log2(marginals[NGRAM] * marginals[TOTAL] ** (cls._n - 1)) - _log2(
            _product(marginals[UNIGRAMS])
        )

```

Que això, expresat amb les igualtats seria:

$$log_{2}(n_{ii} n_{xx}^{(cls.n -1)}) - log_{2} ( n_{ix} n_{xi} )$$

Pointwise Mutual information (PMI) té com a fórmula:
$$PMI(a,b) = log_{2}\frac{P(a,b)}{P(a) P(b)}$$

Si per $PMI csl.n = n-grams$, en aquest cas, bigrams, $csl.n = 2$:

$$log_{2}(n_{ii} n_{xx}^{1}) - log_{2} ( n_{ix} n_{xi} )$$
Operando:
$$log_{2}(\frac{n_{ii} n_{xx}}{n_{ix}n_{xi}})$$

In [40]:
from math import log
x = n_ii * n_xx /( n_ix * n_xi)
x

20.354976737179374

In [41]:
# Hauria de donar --> 6.822
log(x, 2)

4.347309667572599

**CONCLUSIONS**

Succés: la paraula w1 i la paraula w2 apareixen a un text de manera consecutiva.\
Definicions:
- Donat un text, s'analitza tots els bigrams del text. Suma de totes les mostres.
- És fa el recompte del succés (w1, w2).

1- L'estudi dels n-grams té com objectiu detectar parelles de paraules (w1, w2)en el cas de bigrams, que de manera automàtica es detecti com que tenen sentit que juntes tinguin un significat diferent si les analitzem per separat.

2- PMI es basa en la freqüència d'aparició d'aquest succés.



**Exemple**

Suposem que volem detectar ciutats que el nom sigui compost. Si analitzem un text que versi sobre ermites, la ciutat San Sebastian probablement sortirà tant en el cas que es faci referència a la ciutat com al sant.

Ara bé, si el text va de tapes, la ciutat sortirà igualment, però les probabilitats de les paraules per separat seran menors.

Segons la fórmula PMI vol discriminar la freqüència relativa del bigram, és a dir, no acceptem el fet que un bigram sorti molts cops en un text per afirmar que és probable que tingui sentit. La fórmula, en valors de freqüències absolutes, es compon d'un numerador:
$$n_{ii}}$$
Que és el nº de cops que apareix el bigram. En l'exemple tenim que les paraules son etiquetades, i el bigram que més cops es repeteix és (net, sales), el 2n (operating, profit) i el 3er (net, profit). Sembla evident que el text és financer. Quins bigrams ens indica PMI com més probables que tinguin sentit?

In [42]:
print("TERMES CANDIDATS A SER COL·LOCACIONS", b.nbest(bigram_measures.pmi, 3))

TERMES CANDIDATS A SER COL·LOCACIONS [(('0.31', 'CD'), ('emsa', 'NN')), (('03-04', 'JJ'), ('xvi', 'JJ')), (('03.02.2011', 'CD'), ('unimilk', 'JJ'))]


Però si les més freqüents son:

In [43]:
df1.head(3)

,0,1,w1,w1type,w2,w2type
0,"((net, JJ), (sales, NNS))",0.003476,net,JJ,sales,NNS
1,"((operating, NN), (profit, NN))",0.002705,operating,NN,profit,NN
2,"((net, JJ), (profit, NN))",0.001400,net,JJ,profit,NN


Filtrem, eliminem tots els bigrams amb repetició 1, i suposem que no els hi donem cap valor.

In [44]:
b.apply_freq_filter(2)
print("TERMES CANDIDATS A SER COL·LOCACIONS - excloent bigrams freq = 1", b.nbest(bigram_measures.pmi, 3))

TERMES CANDIDATS A SER COL·LOCACIONS - excloent bigrams freq = 1 [(('albat', 'VBP'), ('wirsam', 'JJ')), (('arthritis', 'NN'), ('ra', 'NN')), (('brewers', 'NNS'), ('association', 'VBP'))]


Això passa per que PMI penalitza els bigrams que tinguin paraules que s'utilitzin en altres bigrams. 

Recordem la de Likelihood
```
    def likelihood_ratio(cls, *marginals):
        """Scores ngrams using likelihood ratios as in Manning and Schutze 5.3.4."""
        cont = cls._contingency(*marginals)
        return 2 * sum(
            obs * _ln(obs / (exp + _SMALL) + _SMALL)
            for obs, exp in zip(cont, cls._expected_values(cont))
        )

```

No entenc la fórmula

Al llibre de Maning, (5.10) diu:

$$L(k,n,x) = x^{k}(1-x)^{n-k}$$

Y la fórmula és:

$$logL(c_{12}, c_{1}, p) + logL (c_{2} - c_{12}, N - c_{1}, p) - logL(c_{12}, c_{1}, p_{1}) + logL (c_{2} - c_{12}, N - c_{1}, p_{2})$$

i tenint en compte que la probabilitat que la paraula w2 estigui en el corpus és:

$$p =\frac{c_{2}}{N}$$
$$p_{1} =\frac{c_{12}}{c_{1}}$$
$$p_{2} =\frac{c_{2} - c_{12}}{N - c_{1}}$$

I $c_{1}$ és el nombre de cops de la prarula $w_{1}$, i així succesivament.


